<a href="https://colab.research.google.com/github/SchmetterlingIII/D.T./blob/main/Bracing%20Concept/testing/spline_interp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%autosave 60

Autosaving every 60 seconds


# Spline Interpretation
The function of this .ipynb file is to:

    a. Interpret the curvature of a 2D spline
    b. Do the same with a 3D spline
    c. Do the same with the sensor data (and from now on, only testing with real data rather than abstracted forms as I am nearing the deadline)

### 2D Spline Interpretation

```
a. Plot these lines & interplolations
b. For each segment, get the second derivative and plot them dynamically
```

In [ ]:
import ipywidgets as ipy
import numpy as np
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
from IPython.display import display, Math

In [ ]:
def spline_function(x=1):
    fig, ax = plt.subplots(figsize=(5,5))

    x_list = [-10, 0, x]
    eps = 0.01 # small value so that points aren't stacked on top of each other

    # strictly increasing list
    inc_x_list = []
    for i, val in enumerate(x_list):
        if i == 0:
            inc_x_list.append(val)
        else:
            # make sure each value larger than previous
            inc_x_list.append(max(val, inc_x_list[-1] + eps))

    xs = np.array(inc_x_list)
    ys = np.array([0, 50, 100])

    # I cannot interpret the derivatives of clamped cubic splines
    cs = CubicSpline(xs, ys, bc_type='natural')
    coeffs = []
    for i in range(2):
        for j in range(4):
            coeffs.append(cs.c[j,i])
    # coeffs 0 - 3 are for first segment; 4-7 for second
    display(Math(f"f(x) = {coeffs[0]:.5f}(x + {-1 * xs[0]})^3"))
    print(coeffs)
    print(x_list)

    x_plot = np.linspace(xs[0], xs[-1], 100)

    # plots of f(x), f'(x) and f''(x)
    ax.plot(x_plot, cs(x_plot), alpha=0.8, label='f(x)')
    ax.plot(x_plot, cs(x_plot, 1), label = "f'(x)")
    ax.plot(x_plot, cs(x_plot, 2), label = 'f''(x)')
    ax.scatter(xs, ys, color='k')

    """
    if the gradient of the second derivative is greater than three
    i.e. if the third derivative is greater than three
    """
    second_deriv_linear_coeff = coeffs[0] * 6 # gradient of second derivative of first segment
    if second_deriv_linear_coeff > 3:
        ax.clear()
        # plots of f(x), f'(x) and f''(x)
        ax.plot(x_plot, cs(x_plot), color='k', alpha=1, label='f(x)')
        ax.plot(x_plot, cs(x_plot, 1), label = "f'(x)")
        ax.plot(x_plot, cs(x_plot, 2), label = 'f''(x)')
        ax.scatter(xs, ys, color='k')
        print("Threshold has been exceeded.\nPlot not being displayed.")


    # setup
    ax.grid(alpha=0.25)
    ax.set_xlim(-10.5, 10.5)
    ax.set_ylim(-5, 105)

    plt.show()

In [ ]:
ipy.interact(spline_function, x=(0, 10, 0.05))

### Sensor Test
**Deadline: Tomorrow**

Here I will be using the input of the sensor accelerometer data and doing a similar test: first interpolating using a cubic spline and then reading the second derivative in real time.

This can only be done in Thonny so the results of this are to be recorded on my phone.

**Intended Outcome**:
```
1. Live (noisy) sensor data displayed as vectors, using measured positions as inputs (and designing an algorithm to properly handle that)
2. Cubic spline interpolation of this -- still live and handled well
3. Second derivative display of this data, using feedback from how much I have curved the magnetic sheet to define the thresholds for each area.
4. Some output change (flashing lights on the Arduino) when the threshold has been exceeded
```

**Extension:**
```
- Cleaning up noisy sensor data (or displaying this accumulated noise in a graph).
- Seeing how the spine curves naturally (on the sagittal plane) and getting better results through numerical methods (seeing recordings of people moving their spine or something).
    - This can be developed by me using the IMUs as the beginning for good data collection on this. Therefore, if researchers were then to move on the graphene tubes that I am to research, they would have a good baseline to work from.

```

# Challenge Roadmap: Live Spine Model Prototype

**Task 1**: To transform raw serial data from multiple IMUs into a live, 3D representation of their positions in space.

**Task 2**: Plot these as cubic splines (rather than linear interpolation)

**Task 3**:
- Curvature: use the second derivative of the splines (reading each of the pieces in the colleciton.deque)

- Thresholds
    ```
    for segment in segments:
        if segment_curvature > threshold:
            set timer # using the python timer thing
            if timer > threshold_time:
                arduino_output
                colour_change # for the graph

    ```
- `arduino_output`: sending a command through the serial to the Arduino to change the frequency of the flashing lights


**EXTENSION**:
- math_exchange on understanding *true curvature* -- https://math.stackexchange.com/questions/1457699/easy-derivation-to-show-that-the-curvature-is-kt-frac-alpha-wedge-alpha

- planning how to test this properly at school (and what is left before only doing the aesthetics)
- displaying how noisy the data is and trying to clean it up (having a buffer period for ths):
    - this can be something I write about and explain but never apply since aesthetics and a lo-fi model is more important
- getting this basic script sent off to surgeons en-masse for feedback on how to interpret sections further (and just asking for advice)


`Before Progressing`:

2. annotate the whole code so that I can understand it better for debugging

In [ ]:
import serial.tools.list_ports
import string
import serial
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time
from collections import deque
import numpy as np

# reading the serial data
BAUDRATE = 115200
try:
    # setup for port communication
    ports = serial.tools.list_ports.comports()
    serialInst = serial.Serial()
    portList = [str(i) for i in ports]
    print(portList)

    com = input("Select COM PORT for Arduino: ")

    for i in range(len(portList)):
        if portList[i].startswith("COM" + str(com)):
            SERIAL_PORT = "COM" + str(com)
            print(SERIAL_PORT)

    serialInst.baudrate = BAUDRATE
    serialInst.port = SERIAL_PORT
    serialInst.open()
    print(f"Connected to {SERIAL_PORT} at {BAUDRATE} baud.")

    ## initial setup: 'begin program'
    while True:
        line = serialInst.readline().decode('utf-8') #.strip()
        if line: # if there is data in the readline
            print(f"Arduino: {line}")
        if "Number of sensors: " in line:
           ID_NUM = int(line.strip(":")[-3]) # the number of read sensors
           #print(ID_NUM)
           IMU_DEQUES = [deque(maxlen=50) for i in range(ID_NUM)]
        if "Waiting for 'begin program' command" in line:
            break

    # get the linear distances for the forward kinematics calculation
    print("INSTRUCTIONS:\nInput the linear distances between your sensors in metres.\nMeasure from lowest to highest.\nI would recommend using a high resolution ruler to reduce drift.\n")
    linear_distances = []
    for i in range(ID_NUM - 1):
        value = float(input(f"{i + 1}: "))
        linear_distances.append(value)

    print("Sending 'begin' command to Arduino")
    serialInst.write(b'begin program') # sent in bytes rather than high level strings
                                       # since it is sent to the compiler

    time.sleep(2)

    ###############################################
    """             Data Handling               """
    ###############################################



    # plot setup
    fig = plt.figure()
    ax = fig.add_subplot(projection = '3d')

    ## set the points for the animate(i) function
    ## these variables are what are updated (and saves clearing the plot each time)
    scatter = ax.scatter([], [], [], s=50)
    line, = ax.plot([], [], [])

    ax.set_title("IMU Positions")
    ax.set_xlabel("X (m)")
    ax.set_ylabel("Y (m)")
    ax.set_zlabel("Z (m)")


    # SCALE BY THE LINEAR DISTANCES BETWEEN THE SENSORS
    ### DEFINE FORWARD KINEMATICS FUNCTION
    ### MORE EFFICIENT THAN INITIAL
    ### O(N) << O(N^2)
    def forward_kinematics(matrix):
        # Start with the position of the first sensor (the base)
        if not matrix:
            return []
        positions = [np.array(matrix[0])]

        for i in range(1, len(matrix)):
            direction_vector = np.array(matrix[i])
            distance = linear_distances[i-1]
            link_vector = direction_vector * distance
            new_pos = positions[i-1] + link_vector
            positions.append(new_pos)

        return positions

    ###############################################
    """             Data Plotting               """
    ###############################################

    # the animate function embeds all
    # otherwise more jagged plotting will take place
    def animate(i):
        try:
            IMU_FULL_CHANNEL_DATA = serialInst.readline().decode('utf-8').strip().split(",")
            IMU_ID = int(IMU_FULL_CHANNEL_DATA[-1])
            IMU_DATA = [float(acc) for acc in IMU_FULL_CHANNEL_DATA[:3]] # ONLY APPENDING ACCELERATION, TO CHANGE WHEN DOING KALMAN FILTERING
            IMU_DATA_NORM = IMU_DATA/np.linalg.norm(IMU_DATA)


            if IMU_ID == 1:
                IMU_DEQUES[0].append(IMU_DATA_NORM)

            elif IMU_ID == 2:
                IMU_DEQUES[1].append(IMU_DATA_NORM)

            elif IMU_ID == 7:
                IMU_DEQUES[2].append(IMU_DATA_NORM)

            else:
                print(f"Warning: Recieved data from unknown data channel @ {IMU_ID}")

            if all(IMU_DEQUES):
                IMU_NORMALISED_MATRIX = [dque[-1] for dque in IMU_DEQUES]
                IMU_POSITIONS = np.array(forward_kinematics(IMU_NORMALISED_MATRIX))

                ## update stored variables ("scatter" + "line,") with new positions
                ## more efficient than ax.clear()

                scatter._offsets3d = (IMU_POSITIONS[:, 0], IMU_POSITIONS[:, 1], IMU_POSITIONS[:, 2]) # plots all x, y, z coordinates: regardless of number of rows in this matrix
                line.set_data(IMU_POSITIONS[:, :2].T) # takes in (x,y) values of the points (matplotlib used to only do 2D stuff)
                line.set_3d_properties(IMU_POSITIONS[:, 2]) # add the z-coordinate (3D space) on top of this

        except (ValueError, IndexError) as e:
            print("Serial line error: {e}")
            pass

        return scatter, line,

    # PLOTTING DISPLAY
    anim = FuncAnimation(fig, animate, cache_frame_data=False, interval=200, blit=True)

    ax.set_proj_type('ortho')
    plt.show()

except Exception as e:
    print(f"ERROR: {e}")

### Challenge 3 & 4: Real-time Curvature Analysis & Feedback

**Objective:** To analyze the spline's shape to detect excessive bending and send a feedback signal to the Arduino.

-   **Calculating Curvature from Derivatives:**
    * **Task:** For each spline object, use `cs.derivative(nu=2)` to get the second derivative functions (`x''(t)`, `y''(t)`, `z''(t)`).
    * **Mathematical Challenge:** True curvature ($ \kappa $) is defined by the formula $ \kappa(t)=\frac{\|\mathbf{S}'(t) \times \mathbf{S}''(t)\|}{\|\mathbf{S}'(t)\|^3} $. A powerful and effective proxy for this is the **magnitude of the second derivative vector**:
        $$ \text{Curvature Proxy} = \|\mathbf{S}''(t)\| = \sqrt{(x''(t))^2 + (y''(t))^2 + (z''(t))^2} $$
        Your task is to calculate this value at various points along each segment of your spline in real-time.
    * **Resource:** Your link on deriving curvature (`mathematics.stackexchange.com`) is excellent for understanding the theory. Focus on implementing the proxy formula first, as it directly relates curvature to the second derivatives you can easily compute.

-   **Segment-Specific Thresholding:**
    * **Task:** Create a list or dictionary of threshold values, one for each segment of the spine (e.g., `thresholds = [3.5, 4.0, 4.2, 3.8, 3.5]`).
    * **Challenge:** In your animation loop, check if the calculated curvature proxy for a specific segment exceeds its corresponding threshold.

-   **Arduino Feedback Loop:**
    https://www.youtube.com/watch?v=XXjFtYZEQNw
    * **Task:** When a threshold is breached, use Python's `serial` library to send a specific character (e.g., `'L'`) back to the Arduino.
    * **Challenge:** Modify your Arduino code. It needs to continuously `Serial.read()` to check for incoming data *while* also performing its regular sensor measurements. An `if (Serial.available() > 0)` block will be essential here. When the character `'L'` is received, trigger the onboard LED to flash. This creates the fundamental feedback loop for your haptic motor system later on.